<a href="https://colab.research.google.com/github/JonasZimmer1994/deeplearning_wavenet/blob/master/wavenet_abgabe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/vincentherrmann/pytorch-wavenet.git

Cloning into 'pytorch-wavenet'...
remote: Enumerating objects: 1154, done.
remote: Total 1154 (delta 0), reused 0 (delta 0), pack-reused 1154
Receiving objects: 100% (1154/1154), 268.94 MiB | 32.47 MiB/s, done.
Resolving deltas: 100% (713/713), done.


In [22]:
%cd /content/pytorch-wavenet/

/content/pytorch-wavenet


In [23]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

In [24]:
!pip install tensorflow==1.12.0
import tensorflow as tf
print(tf.__version__)

1.12.0


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-01-08 21:51:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.227.168.133, 3.82.169.244, 34.231.143.21, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.227.168.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  12.8MB/s    in 1.0s    

2020-01-08 21:51:41 (12.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = 'logs/chaconne_model'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://29351b3c.ngrok.io


In [0]:
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
from model_logging import *

In [30]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor
else: 
    print('gpu not available')

gpu not available


In [31]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16,
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model)
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel(
  (filter_convs): ModuleList(
    (0): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (2): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (3): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (4): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (5): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (6): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (7): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (8): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (9): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (10): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (11): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (12): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (13): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (14): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (15): Conv1d (32, 32, kernel_size=(2,), stride=(1,))
    (16): Conv1d (32, 32, kernel_size=(2,), st

In [32]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 598277 items


In [0]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=True)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")

In [0]:
#logger = TensorboardLogger(log_interval=200,
#                           validation_interval=400,
#                           generate_interval=1000,
#                           generate_function=generate_and_log_samples,
#                           log_dir="logs/chaconne_model")

logger = Logger(log_interval=200,
                validation_interval=400,
                generate_interval=1000)

In [0]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=16,
              epochs=10)

start training...
epoch 0


In [0]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)


import IPython.display as ipd

ipd.Audio(generated, rate=16000)